In [ ]:
import re
import json
from aiflow.llm import OpenAI
from aiflow.prompter import (
                        SystemPrompter, 
                        UserPrompter)

gpt = OpenAI(temperature=1)

In [ ]:
german_word = "schwer"
number_of_sentences = 3

In [ ]:
def word_explanator(german_word, llm=gpt):
    system_prompt = SystemPrompter("""
        You are a fluent German speaker that is great at following instructions. 
    """)

    user_prompt = UserPrompter("""
        Given a German word, supply the following
        1. A explanation of the word in english in very simple terms like to a child.
        2. Synonyms of the word if the exist

        Output Format (comma seperated json format):
    
            explanation: <Simple explanation like to a child>
            synonyms: <comman seperated synoyms>

        German Word: {word}
    """)

    response = llm.prompt(messages=[system_prompt(), user_prompt(word=german_word)])
    
    return response

In [ ]:
def sentence_generator(german_word, number_of_sentences, llm=gpt):
    system_prompt = SystemPrompter("""
        You are a fluent German speaker that is great at following instructions. 
    """)

    user_prompt = UserPrompter("""
        Given a German word, generate {number_of_sentences} sentences with the word.
        First provide a contextual scenario in english, then generate the sentence in german.
        The sentences must vary in meaning, conjugated tense, tone (passive or active tone), and quantity (singular or plural) of the word.
        Avoid using the praeteritum instead use partizip II.

        Output Format (comma seperated json format):
        
            context: <A series a sentences in English describing a scenerio for which the generated sentence is application>
            german sentence: <A sentence that was generated based on the supplied german word>
            english translation: <Direct translation of the generated sentence to English>

        German Word: {word}
    """)

    response = llm.prompt(messages=[system_prompt(), user_prompt(word=german_word)])
    
    return response

In [ ]:
def json_parser(response):
    """ parse json response """
    pattern = r'\{[^{}]*\}'
    matches = re.findall(pattern, response)
    data = [json.loads(match) for match in matches]

    return data

In [ ]:
response = word_explanator(german_word)[0]['assistant']
explanation = json_parser(response)
explanation

In [ ]:
response = sentence_generator(german_word, number_of_sentences)[0]['assistant']
sentences = json_parser(response)
sentences

In [ ]:
number_of_sentences